# WES VCF Extract
##### Updated 06/03/2024
##### Tian Yu, Lara Brown
## Note:
This code is not meant to be run on DNANexus and is provided here for convenience. Resources and explanation of how to run are provided at https://github.com/TYTYBU/vcfByGene

#### Goal:
Extract VCF files for exomic regions of given genes from UK BioBank WES data.

#### Required inputs
See https://github.com/TYTYBU/vcfByGene

#### Output
A VCF file for each gene. Output to */selected_genes/hcm/vcf_files*


In [1]:
import pandas as pd

### Set parameters

In [2]:
exon_flank_nt = 5 # flanking nucleotides from the start and end of exons
number_of_threads = 4 # number of threads used in bcftools output compression
tag_str = 'skubali_hcm/vcf_files' # DNAnexus job tag
project_path = 'project-GGy3Bb0JqBj7zfxY8v4by61X:/'
# input vcf path (end with '/')
dx_vcf_path = project_path + "Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants,\ pVCF\ format\ -\ final\ release/"
# output vcf path (end with '/')
dx_vcf_out_path = project_path + "selected_genes/hcm/vcf_files"
# resource path (end with '/')
dx_resource_path = project_path + "GRCh38_resources/"
# difficult regions bed filename
diff_bed = 'GRCh38_alldifficultregions.bed.gz'
# reference genome filename
ref_genome = 'GRCh38_reference_genome.fa' # index filename is inferred
mane_transcript = "./resources/MANE.GRCh38.v1.0.select_ensembl_genomic.csv.gz"
pvcf_block_path = "./resources/pvcf_blocks.txt"


!dx mkdir -p {dx_vcf_out_path} # create gene.vcf output folder

### Helper functions

In [3]:
def get_overlapping_UKB_vcfs(df_gene, df_blk, diagnosis=False):
    vcf_prefix = 'ukb23157_'
    vcf_suffix = '_v1.vcf.gz'
    
    all_vcf_files = []
    df_gene = df_gene.assign(vcf_files = '')
    for index, row in df_gene.iterrows():
        new_vcf_files = df_blk.loc[(df_blk['seqname'] == row['seqname']) & 
                                   ((df_blk['end_pos'] >= row['exon_flank_start']) & (df_blk['start_pos'] <= row['exon_flank_end'])),
                                   'chr_blk_str'].tolist()
        new_vcf_files = list(vcf_prefix + x + vcf_suffix for x in set(new_vcf_files))
        df_gene.loc[index,'vcf_files'] = ','.join(new_vcf_files)
        all_vcf_files = all_vcf_files + new_vcf_files
    all_vcf_files = list(set(all_vcf_files))
    
    if diagnosis:
        return(df_gene)
    else:
        return(all_vcf_files)

### List of gene symbols as input

In [5]:
genes = ["ACTN2", "ALPK3", "DES", "FLNC", "MYBPC3", "MYH6", "MYH7", "PLN", "PTPN11", "TNNI3", "TTR", "TNNT2", "TPM1", "MYL2", "MYL3", "ACTC1"]

### Load pVCF block coordinates

In [6]:
df_blk = pd.read_table(pvcf_block_path, sep = '\t', names = ['ind', 'chr', 'blk', 'start_pos', 'end_pos'])
df_blk['seqname'] = 'chr' + df_blk['chr'].map(str)
df_blk.loc[df_blk['seqname'] == 'chr23', 'seqname'] = 'chrX'
df_blk.loc[df_blk['seqname'] == 'chr24', 'seqname'] = 'chrY'
df_blk['chr_blk_str'] = df_blk['seqname'].str.replace('chr', 'c') + '_b' + df_blk['blk'].map(str)
df_blk

,ind,chr,blk,start_pos,end_pos,seqname,chr_blk_str
0,1,1,0,1,1218130,chr1,c1_b0
1,2,1,1,1218131,1426969,chr1,c1_b1
2,3,1,2,1426970,1758871,chr1,c1_b2
3,4,1,3,1758872,2514221,chr1,c1_b3
4,5,1,4,2514222,3782130,chr1,c1_b4
...,...,...,...,...,...,...,...
972,973,23,20,135552245,141897932,chrX,cX_b20
973,974,23,21,141897933,152168662,chrX,cX_b21
974,975,23,22,152168663,153788223,chrX,cX_b22
975,976,23,23,153788224,156040895,chrX,cX_b23


### Load MANE transcript coordinates

In [7]:
df = pd.read_csv(mane_transcript)
df = df.loc[(df['feature'] == 'exon') & (df['gene_name'].isin(genes))]
df = df[['seqname', 'start', 'end', 'gene_name']]
df['exon_flank_start'] = df['start'] - exon_flank_nt
df['exon_flank_end'] = df['end'] + exon_flank_nt
df['region'] = ((df['seqname'] + ':').str.cat(df['exon_flank_start'].astype(str)) + '-').str.cat(df['exon_flank_end'].astype(str))
df

,seqname,start,end,gene_name,exon_flank_start,exon_flank_end,region
21285,chr1,26067336,26067630,TRIM63,26067331,26067635,chr1:26067331-26067635
21288,chr1,26066268,26066440,TRIM63,26066263,26066445,chr1:26066263-26066445
21290,chr1,26061166,26061334,TRIM63,26061161,26061339,chr1:26061161-26061339
21292,chr1,26060266,26060361,TRIM63,26060261,26060366,chr1:26060261-26060366
21294,chr1,26058390,26058623,TRIM63,26058385,26058628,chr1:26058385-26058628
...,...,...,...,...,...,...,...
473557,chr20,44159618,44160407,JPH2,44159613,44160412,chr20:44159613-44160412
473559,chr20,44118505,44118623,JPH2,44118500,44118628,chr20:44118500-44118628
473561,chr20,44115665,44116386,JPH2,44115660,44116391,chr20:44115660-44116391
473563,chr20,44114782,44114876,JPH2,44114777,44114881,chr20:44114777-44114881


### Run Swiss-army-knife on DNAnexus
- get region info for each gene from block file
- bcftools command for step 2 finished
- bcftools command for step 3 & 4 TBD
- list genes not included in MANE set

In [ ]:
genes_not_found = []
genes_found = []
known_large_genes = ["DSP", "TSC2", "TTN", "NCOA3"]

for gene in genes:
    df_gene = df.loc[df['gene_name'] == gene]
    if df_gene.shape[0] > 0:
        genes_found.append(gene)
        
        vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
        vcf_str = ' '.join(vcf_files)
        region_str = ','.join(df_gene['region'].to_list())
        
        if ((len(vcf_files) > 1) or (gene in known_large_genes)):
            mem_level = "mem2_ssd1_v2_x16" # dynamically change memory level when submitting jobs on DNAnexus
        else:
            mem_level = "mem1_ssd1_v2_x4"
        
        # filtering variants
        bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
        bcftools_cmd2 = "bcftools view -Ou --max-alleles 5 -T ^" + diff_bed
        bcftools_cmd3 = "bcftools +fill-tags -Ou -- -t all"
        bcftools_cmd4 = "bcftools norm -Ou -m - -f " + ref_genome 
        bcftools_cmd5 = "bcftools view -Oz -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + "_variants.vcf.gz"
        
        # extract carriers
        bcftools_cmd6 = "mkdir -p carrier"
        bcftools_cmd7 = "bcftools query -i 'GT=\\\"RA\\\"|GT=\\\"AR\\\"|GT=\\\"AA\\\"' -f '%CHROM  %POS %REF %ALT %INFO/AF [%SAMPLE|]\n' " + gene + "_variants.vcf.gz > carrier/" + gene + ".ssv"
        
        # parsing command
        bcftools_command_a = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])
        bcftools_command_b = " && ".join([bcftools_cmd6, bcftools_cmd7])
        bcftools_command = bcftools_command_a + " && " + bcftools_command_b
        
        # parsing input
        
        dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))

        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'

        # final dx command
        dx_command = 'dx run swiss-army-knife --instance-type ' + mem_level + ' -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene
        !{dx_command}
    else:
        genes_not_found.append(gene)
            
print('Genes not found in MANE database:')
print(genes_not_found)